# Keras R&D

In [ ]:
import re
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
TEXT = '''
In a quiet village, a curious boy named Liam had a loyal horse named Blaze.
Together, they discovered an ancient device that projected holograms of the human brain, illustrating the wonders of machine learning.
One night, while experimenting, the device activated, propelling them into the future.
In this world, machines and humans coexisted harmoniously, their minds intertwined through advanced AI.

Liam learned that this future was shaped by understanding the brain's intricacies.
As they time-traveled back, armed with knowledge, they set out to share their discoveries, ensuring a future where technology and humanity thrived together,
guided by wisdom and compassion.

In a quiet village nestled between rolling hills and dense forests, a boy named Liam lived with his parents. His best friend and constant companion was a majestic horse named Blaze, with a coat as black as night and eyes that sparkled with intelligence. Liam had always been fascinated by the mysteries of the human mind and the promise of technology. His father, a scientist, had instilled in him a deep love for discovery and innovation.

One crisp autumn day, while exploring the outskirts of the village, Liam stumbled upon a hidden cave. Inside, he found a peculiar device covered in dust and cobwebs. The device had a series of buttons and a holographic projector. Intrigued, Liam brought it back to his father, who recognized it as an ancient relic from a long-forgotten civilization. This device, his father explained, could project detailed images of the human brain and was rumored to have the ability to manipulate time.

Liam and his father spent weeks studying the device, trying to unlock its secrets. One night, while experimenting with different settings, they accidentally activated it. A blinding light enveloped Liam and Blaze, and they were transported through a vortex of time and space. When the light faded, they found themselves in a world unlike anything they had ever seen.

They had arrived in a future where technology and humanity had evolved together in perfect harmony. Skyscrapers touched the clouds, and advanced machines coexisted with humans, their minds connected through neural interfaces powered by sophisticated AI. People moved about with an air of serenity and purpose, their lives enriched by the seamless integration of technology.

Liam was fascinated by this future. He learned that the advancements were made possible by a profound understanding of the human brain's intricacies. Scientists had used machine learning to decode the brain's mysteries, creating AI that could augment human abilities and foster unprecedented innovation. This future society had eradicated diseases, solved complex global issues, and achieved a level of prosperity and happiness that Liam had never imagined.

During their stay, Liam and Blaze met a wise old scientist named Dr. Elena. She explained that this harmonious future was not always guaranteed. It was the result of generations of effort, guided by the lessons learned from the past. Dr. Elena showed Liam how they used machine learning to predict and prevent potential crises, ensuring a stable and prosperous world.

Inspired by what he had learned, Liam realized the importance of understanding the human brain and harnessing the power of technology for good. Dr. Elena offered them a chance to return to their own time, armed with the knowledge they had gained. With a heavy heart but a sense of purpose, Liam and Blaze stepped back into the vortex, ready to share their discoveries.

Upon returning to their village, Liam and his father began working tirelessly to apply the insights from the future. They developed new ways to study the brain, creating early versions of neural interfaces and AI-driven solutions to improve people's lives. The villagers were initially skeptical, but they soon saw the benefits of these innovations.

Years passed, and Liam grew into a wise and respected scientist, much like Dr. Elena. Blaze remained by his side, a symbol of their incredible journey. Together, they inspired a new generation of thinkers and dreamers, ensuring that their world would one day achieve the harmonious future they had glimpsed.

Liam never forgot the lessons he learned during his time travel adventure. He knew that the key to a brighter future lay in understanding the past, embracing the present, and always striving for progress with wisdom and compassion. As he looked out over the village, now a thriving hub of innovation, he knew that he and Blaze had set the wheels of destiny in motion, creating a legacy that would endure for generations to come.
'''

In [ ]:
class MyTextCleaner(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(MyTextCleaner, self).__init__(**kwargs)

    def call(self, inputs):
        txt = tf.strings.lower(inputs)
        txt = tf.strings.regex_replace(txt, r'[\.\,\n]', ' ')
        txt = tf.strings.regex_replace(txt, r'\b(.*?)\-(.*?)\b', r'\1 <HYPHEN> \2')
        txt = tf.strings.regex_replace(txt, r'\b(.*?)\'s\b', r'\1 <APOSTROPHE-S>')
        txt = tf.strings.regex_replace(txt, r'\s\s+', ' ')
        txt = tf.strings.strip(txt)
        return txt

In [ ]:
def process_text(txt: tf.Tensor) -> tf.Tensor:
    txt = tf.strings.lower(txt)
    txt = tf.strings.regex_replace(txt, r'[\.\,\n]', ' ')
    txt = tf.strings.regex_replace(txt, r'\b(.*?)\-(.*?)\b', r'\1 <HYPHEN> \2')
    txt = tf.strings.regex_replace(txt, r'\b(.*?)\'s\b', r'\1 <APOSTROPHE-S>')
    txt = tf.strings.regex_replace(txt, r'\s\s+', ' ')
    txt = tf.strings.strip(txt)

    return txt

In [ ]:
VOCAB = {}
txt_cleaner = MyTextCleaner()
for line in TEXT.strip().split('\n'):
    if line.strip() == '':
        continue

    line = tf.constant(line, dtype=tf.string)
    # tokens = process_text(line).numpy().decode('utf-8').split(' ')
    tokens = txt_cleaner(line).numpy().decode('utf-8').split(' ')
    for val in tokens:
        VOCAB[val] = VOCAB.get(val, 0) + 1

VOCAB = {k: v for k, v in sorted(VOCAB.items(), key=lambda v: (v[1], v[0]), reverse=True)}

In [ ]:
for i, v in enumerate(VOCAB.keys()):
    print(f'{i+2:>3} {v:<16}', end=' ')
    if (i + 1) % 7 == 0:
        print('')

In [ ]:
sample_input = tf.constant([[' the brain of ai is, over-time ']], dtype=tf.string)

In [ ]:
process_text(sample_input)

In [ ]:
vec = layers.TextVectorization(
    standardize=process_text,
    output_mode='int',
    output_sequence_length=16,
    vocabulary=list(VOCAB.keys()),
)

In [ ]:
vec(sample_input)

In [ ]:
def build_my_model():
    layer_input = layers.Input(shape=[1], dtype=tf.string)

    vec = layers.TextVectorization(
        standardize=process_text,
        output_mode='int',
        output_sequence_length=16,
        vocabulary=list(VOCAB.keys()),
        name='Vec'
    )
    embedding = layers.Embedding(input_dim=len(VOCAB) + 2, output_dim=4)
    dense = layers.Dense(8, activation='relu')
    output_layer = layers.Dense(1, activation='sigmoid')

    x = vec(layer_input)
    x = embedding(x)
    x = tf.reduce_mean(x, axis=1)
    x = dense(x)
    x = output_layer(x)

    return tf.keras.Model(inputs=layer_input, outputs=x)

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab, version):
        super(MyModel, self).__init__()
        self.txt_cleaner = MyTextCleaner()
        self.vec = layers.TextVectorization(
            standardize=None,
            output_mode='int',
            output_sequence_length=16,
            vocabulary=list(VOCAB.keys()),
        )
        self.embedding = layers.Embedding(input_dim=len(VOCAB) + 2, output_dim=4)
        self.dense = layers.Dense(8, activation='relu')
        self.output_layer = layers.Dense(1, activation='sigmoid')

        self.embedding_output = None
        self.vec_output = None
        self.txt_post_output = None
        self.vocab = vocab
        self._version = version

    # def get_config(self):
    #     config = super(MyModel, self).get_config()
        # config.update({'version': self._version})
        # return config

    def call(self, inputs, return_embedding_output=False, return_vec_output=False, return_txt_output=False):
        x = self.txt_cleaner(inputs)
        if return_vec_output:
            self.txt_post_output = x
        else:
            self.txt_post_output = None

        x = self.vec(x)
        if return_vec_output:
            self.vec_output = x
        else:
            self.vec_output = None

        x = self.embedding(x)
        if return_embedding_output:
            self.embedding_output = x
        else:
            self.embedding_output = None
        x = tf.reduce_mean(x, axis=1)
        x = self.dense(x)
        x = self.output_layer(x)
        return x

    def get_embedding_output(self):
        return self.embedding_output

    def get_vec_output(self):
        return self.vec_output

    def get_txt_output(self):
        return self.txt_post_output

    def get_vocab(self):
        return self.vocab

    def get_version(self):
        return self._version

In [ ]:
# my_model = build_my_model()
# my_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# my_model.summary()

# tf.keras.utils.plot_model(my_model, show_shapes=True, show_dtype=True, expand_nested=True)

In [ ]:
vocab = tf.constant(['<EMPTY>', '<OOV>'] + [k for k, v in VOCAB.items()])
model = MyModel(vocab, tf.constant('1.0.2-dev', dtype=tf.string))
model(sample_input, True, True, True)

In [ ]:
model.get_embedding_output()

In [ ]:
model.get_vec_output()

In [ ]:
model.get_txt_output()

In [ ]:
model.get_version()

In [ ]:
model.get_vocab()[:32]

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit([['the book'], ['the brain is of time']], [1, 0])

In [ ]:
@tf.function(input_signature=[tf.TensorSpec(shape=[None, 1], dtype=tf.string)])
def serve_model(inputs):
    return {"outputs": model(inputs), 'version': model.get_version()}

@tf.function(input_signature=[tf.TensorSpec(shape=[None, 1], dtype=tf.string)])
def serve_embedding(inputs):
    model(inputs, True)
    return {"embedding_output": model.get_embedding_output()}

@tf.function(input_signature=[tf.TensorSpec(shape=[None, 1], dtype=tf.string)])
def serve_vec(inputs):
    model(inputs, False, True)
    return {"vec_output": model.get_vec_output()}

@tf.function(input_signature=[tf.TensorSpec(shape=[None, 1], dtype=tf.string)])
def serve_txt(inputs):
    model(inputs, False, True)
    return {"txt_output": model.get_txt_output()}

@tf.function(input_signature=[tf.TensorSpec(shape=[None, 1], dtype=tf.string)])
def serve_vocab(inputs):
    model(inputs, False, True)
    return {"vocab": model.get_vocab()}

@tf.function(input_signature=[tf.TensorSpec(shape=[None, 1], dtype=tf.string)])
def serve_guts(inputs):
    model(inputs, True, True)
    return {
        "embedding_output": model.get_embedding_output(),
        "vec_output": model.get_vec_output(),
        'txt_output': model.get_txt_output(),
        'vocab': model.get_vocab(),
    }

In [ ]:
model.save('saved_models/gutsy-model', signatures={
    "serving_default": serve_model,
    "embedding": serve_embedding,
    "vec": serve_embedding,
    "txt": serve_txt,
    "vocab": serve_vocab,
    "guts": serve_guts,
})

## Load saved model

In [1]:
import tensorflow as tf

2024-05-26 05:36:00.626044: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-26 05:36:01.432353: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
sample_input = tf.constant([
    [' The Brain Of AI AI. is, over-time '],
    ['Once upon a time there\'s a mid-century BRAIN from a lot of over-time that he\'s not there'],
], dtype=tf.string)

2024-05-26 05:36:02.216793: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-05-26 05:36:02.216837: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: ml03
2024-05-26 05:36:02.216844: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: ml03
2024-05-26 05:36:02.216892: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 460.106.0
2024-05-26 05:36:02.216912: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 460.106.0
2024-05-26 05:36:02.216917: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 460.106.0


In [3]:
new_model = tf.saved_model.load('saved_models/gutsy-model')

In [4]:
new_model.signatures

_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper_serve_model(*, inputs) at 0x7F55F04353A0>, 'embedding': <ConcreteFunction signature_wrapper_serve_embedding(*, inputs) at 0x7F55F03FD2B0>, 'vec': <ConcreteFunction signature_wrapper_serve_embedding(*, inputs) at 0x7F55F0364820>, 'txt': <ConcreteFunction signature_wrapper_serve_txt(*, inputs) at 0x7F55F036F1C0>, 'vocab': <ConcreteFunction signature_wrapper_serve_vocab(*, inputs) at 0x7F55F4346130>, 'guts': <ConcreteFunction signature_wrapper_serve_guts(*, inputs) at 0x7F55F03FAA60>})

In [5]:
f = new_model.signatures['serving_default']

In [6]:
f(sample_input)

{'version': <tf.Tensor: shape=(), dtype=string, numpy=b'1.0.2-dev'>,
 'outputs': <tf.Tensor: shape=(2, 1), dtype=float32, numpy=
 array([[0.49591082],
        [0.49807087]], dtype=float32)>}

In [7]:
g = new_model.signatures['guts']
sample = g(sample_input)

In [8]:
for txt, vec in zip(sample['txt_output'].numpy(), sample['vec_output'].numpy()):
    # txt = txt[0].decode('utf-8').split(' ')
    print(
        ' '.join(
            [f'{i:<8}' for i in txt[0].decode('utf-8').split()]
        )
    )
    dec = ' '.join([
        f"{sample['vocab'].numpy()[i].decode('utf-8'):<8}" for i in vec
    ])
    print(dec)
    print('-' * 32)

the      brain    of       ai       ai       is       over     <HYPHEN> time    
the      brain    of       ai       ai       <OOV>    over     <HYPHEN> time     <EMPTY>  <EMPTY>  <EMPTY>  <EMPTY>  <EMPTY>  <EMPTY>  <EMPTY> 
--------------------------------
once     upon     a        time     there    <APOSTROPHE-S> a        mid      <HYPHEN> century  brain    from     a        lot      of       over     <HYPHEN> time     that     he       <APOSTROPHE-S> not      there   
<OOV>    upon     a        time     <OOV>    <APOSTROPHE-S> a        <OOV>    <HYPHEN> <OOV>    brain    from     a        <OOV>    of       over    
--------------------------------
